In beginning to clean up the code, as mentioned Mary Lauren began to clean up the code initially by creating a data path to the data, and following by reading in each of the data sets given the data path and file name. From there, 

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import csv
data_path = "/Users/geenaildefonso/Downloads/DataP/"

In [13]:
###
#   Mary Lauren Benton, 2017
###

###
#   imports & variables
###

df_adt = pd.read_csv(data_path+"df_adt_clean.csv", parse_dates=['admission_date', 'discharge_date'], index_col=0)



###
#   ADT
### 
# adt = pd.read_csv(data_path+"FONNESBECK_ADT.csv", na_values=[''], 
#                   parse_dates=['Admission_date', 'Event_Date', 'DISCHARGE_DATE'],
#                   encoding = "ISO-8859-1")
# adt.head()

# # rename the columns and replace event strings with simpler versions
# # #todo -- expand categorical variables using get_dummies
# adt_clean = (adt
#              .rename(columns={"RUID": "patient_id", 
#                               "Event":"adt_event", 
#                               "Admission_date": "admission_date",
#                               "Event_Date": "adt_event_date", 
#                               "SRV_CODE": "srv_code",
#                               "CHIEF_COMPLAINT": "chief_complaint", 
#                               "DISCHARGE_DATE": "discharge_date"})
#              .replace({'adt_event': {'.*Admit': 'admit',
#                                      '.*Discharge': 'discharge', 
#                                      '.*Transfer': 'transfer'}}, regex=True))

# adt_clean.head()

# calculate the amount of missing data in the ADT table
# df_adt.isnull().sum()

# presumably only discharges will have discharge dates; these actual missing data
# #todo -- decide how to handle missing dates
# (df_adt[df_adt.adt_event == 'discharge']).isnull().sum()

In [14]:
df = pd.DataFrame(adt_clean, columns = ['patient_id','adt_event','admission_date', 'discharge_date'])
df = df[df.adt_event != 'transfer']
df = df[df.adt_event != 'discharge']
# adt_final
df_adt = df[['patient_id','admission_date', 'discharge_date']]
df_adt = df_adt.sort_values('admission_date')
df_adt = df_adt.sort_values('patient_id')
df_adt.head()

NameError: name 'adt_clean' is not defined

In [104]:
###
#   BMI
###
bmi = pd.read_csv(data_path+"df_bmi_clean.csv", parse_dates=['bmi_date'], infer_datetime_format=True)
# bmi.head()

# bmi_clean = (bmi
#              .rename(columns={"RUID": "patient_id", 
#                               "BMI": "bmi",
#                               "Date_BMI": "bmi_date", 
#                               "BMI_Weight": "weight",
#                               "BMI_Height": "height", 
#                               "Pregnancy_Indicator": "pregnant"}))
bmi_clean['bmi_date'] = pd.to_datetime(bmi_clean.bmi_date, errors='coerce')
# small amount of missingness
# #todo -- possible to fill in missing if same patient
# bmi_clean.isnull().sum()
df_bmi = bmi_clean.drop_duplicates(subset='bmi_date')
df_bmi = df_bmi[['patient_id', 'bmi', 'bmi_date']].dropna()
# df_bmi.head()
# df_bmi.to_csv('/Users/geenaildefonso/Downloads/DataP/df_bmi_clean.csv')
# df_bmi = df_bmi.sort_values('bmi_date')
# df_bmi.head()
# df_bmi = df_bmi.sort_values('patient_id', inplace=True)
# df_bmi = df_bmi.sort(df_bmi.bmi_date.tolist())
# for i in x['patient_id']:
#     if x['bmi_date'] = 
        
# df = bmi_clean.groupby('bmi_date',sort=True).sum()
# df_bmi[df_bmi.patient_id== 50135361]

In [98]:
import statsmodels.formula.api as smf
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

unique_ids = df_bmi.patient_id.unique()
ps_means = list(range(len(unique_ids)))
ps_std = list(range(len(unique_ids)))
ps_trends = list(range(len(unique_ids)))
# pd_trends = list(range(len(unique_ids)))
for p, count in zip(unique_ids, range(len(unique_ids))):
#make a subsetted dataframe of just that ID
    p_df = df_bmi[df_bmi.patient_id == p]
    p_df=p_df.reset_index(drop=True)
    #for each measurement in that list:
    s_mean=p_df.bmi.mean()
    s_std=p_df.bmi.std()
#     d_mean=p_df.diastolic.mean()
    ps_means[count]=s_mean
    ps_std[count] = s_std
#     pd_means[count]=d_mean
    #Trends in dp and sp: fit each patient's data with a linear regression and record the slope.
    d_regr = linear_model.LinearRegression()
    d_regr.fit(p_df.bmi_date.reshape(-1, 1), p_df.bmi.reshape(-1, 1))
    b=d_regr.coef_.item(0)
    ps_trends[count]=b

x_bmi = pd.DataFrame({'patient_id':unique_ids, 
                     'ps_mean': ps_means,'ps_std': ps_std})

/Users/geenaildefonso/anaconda/envs/bios8366/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [99]:
x_bmi.head()

,patient_id,ps_mean,ps_std
0,50135262,43.443846,4.362163
1,50135361,40.104062,56.286141
2,50135369,29.091786,26.354793
3,50135375,26.887687,2.139820
4,50135425,33.648269,5.109400


In [24]:
df_bmi.bmi_date.min()

Timestamp('1994-07-05 00:00:00')

In [72]:
###
#   BP
###
bp = pd.read_csv(data_path+"FONNESBECK_BP.csv", parse_dates=['Measure_date'], infer_datetime_format=True)
bp.head()

bp_clean = (bp
            .rename(columns={"RUID": "patient_id", 
                             "SYSTOLIC": "systolic",
                             "DIASTOLIC": "diastolic", 
                             "Measure_date": "bp_date"}))
bp_clean['bp_date'] = pd.to_datetime(bp_clean.bp_date, errors='coerce')
bp_clean.head()

# only missing dates, may not need to address
# bp_clean.isnull().sum()

,patient_id,systolic,diastolic,bp_date
0,50135262,150,80,2005-01-09
1,50135262,137,77,2007-02-08
2,50135262,137,78,2007-02-08
3,50135262,190,78,2007-02-08
4,50135262,190,77,2007-02-08


In [57]:
###
#   CPT
###
cpt = pd.read_csv(data_path+"FONNESBECK_CPT.csv", parse_dates=['Event_date'], infer_datetime_format=True)
cpt.head()

cpt_clean = (cpt
             .rename(columns={"RUID": "patient_id", 
                              "CPT_Code": "cpt_code",
                              "Event_date": "cpt_event_date"}))
cpt_clean.to_csv('/Users/geenaildefonso/Downloads/DataP/df_cpt_clean.csv')
cpt_clean.head()

# no missing data
# cpt_clean.isnull().sum()
cpt_clean.head()

,patient_id,cpt_code,cpt_event_date
0,50135262,00001,2004-11-28
1,50135262,00005,2012-05-03
2,50135262,00025,2007-02-08
3,50135262,00025,2004-11-16
4,50135262,00030,2007-02-08
5,50135262,00030,2007-02-09
6,50135262,00030,2011-02-11
7,50135262,00030,2007-02-12
8,50135262,00030,2007-02-15
9,50135262,00030,2007-02-16


In [105]:
###
#   EGFR
###
egfr = pd.read_csv(data_path+"df_egfr_clean.csv", parse_dates=['egfr_date'], infer_datetime_format=True)

# egfr_clean = (egfr
#               .rename(columns={"RUID": "patient_id", 
#                                "EGFR": "egfr"}))
# egfr.to_csv('/Users/geenaildefonso/Downloads/DataP/df_egfr_clean.csv')
df_egfr = egfr_clean[['patient_id', 'egfr', 'egfr_date']]
# df_egfr = df_egfr.set_index(['patient_id'])
df_egfr.head()
# df_egfr.index.unique()# no missing data
# egfr.isnull().sum()

,patient_id,egfr,egfr_date
0,50135262,123.68000,2007-02-08
1,50135262,123.67783,2007-02-08
2,50135262,76.40173,2011-02-11
3,50135262,76.40000,2011-02-11
4,50135262,78.64000,2011-02-12


In [106]:
unique_ids = df_egfr.patient_id.unique()
ps_means = list(range(len(unique_ids)))
ps_std = list(range(len(unique_ids)))
ps_trends = list(range(len(unique_ids)))
# pd_trends = list(range(len(unique_ids)))
for p, count in zip(unique_ids, range(len(unique_ids))):
#make a subsetted dataframe of just that ID
    p_df = df_egfr[df_egfr.patient_id == p]
    p_df=p_df.reset_index(drop=True)
    #for each measurement in that list:
    s_mean=p_df.egfr.mean()
    s_std=p_df.egfr.std()
#     d_mean=p_df.diastolic.mean()
    ps_means[count]=s_mean
    ps_std[count] = s_std
#     pd_means[count]=d_mean
    #Trends in dp and sp: fit each patient's data with a linear regression and record the slope.
    d_regr = linear_model.LinearRegression()
    d_regr.fit(p_df.egfr_date.reshape(-1, 1), p_df.egfr.reshape(-1, 1))
    b=d_regr.coef_.item(0)
    ps_trends[count]=b

x_egfr = pd.DataFrame({'patient_id':unique_ids, 
                     'ps_mean': ps_means,'ps_std': ps_std})

/Users/geenaildefonso/anaconda/envs/bios8366/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [107]:
x_egfr.head()

,patient_id,ps_mean,ps_std
0,50135262,87.336092,24.538705
1,50135361,51.882929,18.630883
2,50135369,84.715450,14.981761
3,50135375,35.632098,19.640734
4,50135425,35.230333,32.324784


In [55]:
###
#   ICD9
###
icd9 = pd.read_csv(data_path+"FONNESBECK_ICD9.csv", parse_dates=['Event_date'], infer_datetime_format=True)
icd9.head()

icd9_clean = (icd9
              .rename(columns={"RUID": "patient_id", 
                               "ICD9_Code": "icd9_code",
                               "Event_date": "icd9_event_date"}))
icd9_clean.to_csv('/Users/geenaildefonso/Downloads/DataP/df_icd9_clean.csv')
icd9_clean.head()

# no missing data
icd9_clean.isnull().sum()


patient_id         0
icd9_code          1
icd9_event_date    0
dtype: int64

In [54]:
###
#   LAB
###
lab = pd.read_csv(data_path+"FONNESBECK_LAB.csv", parse_dates=['Lab_date'], 
                  infer_datetime_format=True, quoting=csv.QUOTE_NONE, na_values=['>'])
lab.head()

lab_clean = (lab
             .rename(columns={"RUID": "patient_id", 
                              "Lab_name": "short_lab_name",
                              "Lab_date": "lab_date", 
                              "Lab_value": "lab_value"}))
lab_clean.head()

# decent number of missing lab values
# may be able to impute missing values if same patient 
lab_clean.isnull().sum()
lab_clean.to_csv('/Users/geenaildefonso/Downloads/DataP/lab_clean_clean.csv')

In [56]:
###
#   MED
###
med = pd.read_csv(data_path+"df_med_clean.csv", parse_dates=['drug_entry_date'], infer_datetime_format=True)
# med

# med_clean = (med
#              .rename(columns={"RUID": "patient_id", 
#                               "Entry_Date": "drug_entry_date",
#                               "Drug_Name": "drug_name", 
#                               "DRUG_FORM": "drug_form",
#                               "DRUG_STRENGTH": "drug_strength",
#                               "Route": "drug_route",
#                               "Dose_Amt": "drug_dose",
#                               "Drug_Freq": "drug_freq",
#                               "Duration": "drug_duration"}))
# med_clean.head()

# lots of missing data in this table
# many cols likely uninformative; drug_name might be most useful
# med_clean.isnull().sum()
med = med[['patient_id', 'drug_entry_date', 'drug_name']]
med
# med_clean

,patient_id,drug_entry_date,drug_name
0,50135262,2011-02-13,ACETAMINOPHEN : TYLENOL
1,50135262,2012-05-23,ACETAMINOPHEN : TYLENOL
2,50135262,2012-05-23,ACETAMINOPHEN : TYLENOL
3,50135262,2010-10-29,ACETAMINOPHEN : TYLENOL
4,50135262,2010-10-29,ACETAMINOPHEN : TYLENOL
5,50135262,2011-02-11,ACETAMINOPHEN ORAL SOLUTION
6,50135262,2008-02-26,ACETAMINOPHEN: TYLENOL
7,50135262,2012-05-02,ACETAMINOPHEN: TYLENOL
8,50135262,2014-11-15,ACETAMINOPHEN: TYLENOL
9,50135262,2004-11-16,ACETAMINOPHEN: TYLENOL


In [53]:

###
#   PHENOTYPE
###
phenotype = pd.read_csv(data_path+"FONNESBECK_phenotype.csv", parse_dates=['DOB', 'DOD'], infer_datetime_format=True)
# print(phenotype.head())

phenotype_clean = (phenotype
                   .rename(columns={"RUID": "patient_id", 
                                    "Sex": "sex",
                                    "DOB": "DOB", 
                                    "DOD": "DOD",
                                    "Race": "race"})
                   .replace({'sex':  {'F': 0,'M': 1, 'U': 'NaN', '.': 'NaN', 'NA': 'NaN'}}))
phenotype_clean.head()

# lots of missing DOBs and sex
# living patients will have missing DOD
# #todo -- decide whether to collapse race into fewer categories
# phenotype_clean.isnull().sum()
# phenotype_clean.head()
df_pheno = phenotype_clean[['patient_id','sex', 'DOB', 'race']]
df_pheno.to_csv('/Users/geenaildefonso/Downloads/DataP/df_pheno_clean.csv')